In [1]:
import json
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD, RMSprop
import preprocess
import model  # import your model


2025-06-30 05:25:48.597132: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 05:25:48.709947: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 05:25:49.914636: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-30 05:25:50.456334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751261150.989723    1042 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751261151.17

In [ ]:
def get_model(name, input_shape, num_classes, dropout):
    if name == "model":
        return model.build_model(input_shape=input_shape, num_classes=num_classes, dropout=dropout)
    else:
        raise ValueError("Unknown model name.")

def train(model, config, train_ds, val_ds, name):
    model.compile(
        optimizer=Adam(learning_rate=config["learning_rate"]),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=config["epochs"]
    )
    model.save(f"models/{name}_model.h5")
    print(f"{name} model saved at models/{name}_model.h5!")
    return history


In [4]:

model_name = "model"  

# Load config from file (change path accordingly)
config_path = "../../config/config1.json"
with open(config_path) as f:
    config = json.load(f)

input_shape = (48, 48, 1)
num_classes = 7

# Load datasets
train_ds, val_ds, _ = preprocess.get_data_splits(img_size=(48, 48), batch_size=config["batch_size"])

# Apply augmentation on training data
augment = preprocess.get_augmentation_model(input_shape=input_shape)
train_ds = train_ds.map(lambda x, y: (augment(x, training=True), y))


Found 28821 files belonging to 7 classes.


2025-06-29 15:11:59.687134: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 7066 files belonging to 7 classes.


/opt/conda/envs/ml-env/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
model = get_model(model_name, input_shape, num_classes, config.get("dropout", 0.5))
#  Train the model and return training history
history = train(model, config, train_ds, val_ds, model_name)


/opt/conda/envs/ml-env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 282s 307ms/step - accuracy: 0.2095 - loss: 2.1348 - val_accuracy: 8.4914e-04 - val_loss: 154.7178
Epoch 2/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 269s 298ms/step - accuracy: 0.2594 - loss: 1.7987 - val_accuracy: 0.0000e+00 - val_loss: 168.5153
Epoch 3/20
312/901 ━━━━━━━━━━━━━━━━━━━━ 2:51 290ms/step - accuracy: 0.2955 - loss: 1.7311